In [1]:
import cv2


In [ ]:
import cv2
import os
import numpy as np
from PIL import Image

def generate_dataset(user_id, user_name, samples=50, save_dir="data"):
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    if face_classifier.empty():
        raise Exception("Error: Haar cascade not loaded.")

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Train a temporary recognizer on existing dataset to detect duplicates
    existing_faces = []
    existing_ids = []
    for file in os.listdir(save_dir):
        if file.endswith(".jpg"):
            path = os.path.join(save_dir, file)
            img = Image.open(path).convert('L')
            img_np = np.array(img, 'uint8')
            id_in_file = int(file.split(".")[1])
            existing_faces.append(img_np)
            existing_ids.append(id_in_file)
    
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    if len(existing_faces) > 0:
        recognizer.train(existing_faces, np.array(existing_ids))
    else:
        recognizer = None  # No previous faces yet

    def face_cropped(img):
        if img is None:
            return None
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            return img[y:y+h, x:x+w], (x, y, w, h)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise Exception("Error: Could not open webcam.")

    img_id = 0
    print(f"Collecting samples for {user_name}. Press 'q' to stop early.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame from webcam.")
            break

        result = face_cropped(frame)
        if result is not None:
            face, (x, y, w, h) = result
            gray_face = cv2.cvtColor(cv2.resize(face, (200, 200)), cv2.COLOR_BGR2GRAY)

            # Check if face already exists in dataset
            if recognizer is not None:
                try:
                    pred_id, conf = recognizer.predict(gray_face)
                    confidence = int(100 * (1 - conf / 300))
                    if confidence > 95:  # Face already exists
                        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 2)
                        cv2.putText(frame, f"{user_name} cannot take pics! Already exists",
                                    (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
                        cv2.imshow("Capturing Faces", frame)
                        cv2.waitKey(2000)  # show message for 2 sec
                        break
                except:
                    pass  # No match found

            # Save new image
            img_id += 1
            file_name_path = f"{save_dir}/{user_name}.{user_id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, gray_face)

            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f"{user_name} ({img_id}/{samples})", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.imshow("Capturing Faces", frame)

        if cv2.waitKey(1) & 0xFF == ord('q') or img_id >= samples:
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Collected {img_id} samples for {user_name} successfully!")

# Example usage
generate_dataset(user_id=1, user_name="Alishba", samples=50)
generate_dataset(user_id=2, user_name="Kinza", samples=50)



Train the classifier and save it

In [24]:
import os
import cv2
from PIL import Image #pip install pillow
import numpy as np    # pip install numpy

In [72]:
import cv2
import os
import numpy as np
from PIL import Image

def train_classifier(data_dir="data"):
    """
    Train LBPH face recognizer on images stored in `data_dir`.

    Each image file should follow the format: name.user_id.sample_id.jpg
    Example: Alishba.1.1.jpg
    """
    faces = []
    ids = []

    image_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".jpg")]

    if len(image_files) == 0:
        print("No images found in the data directory.")
        return

    for image_path in image_files:
        try:
            # Convert to grayscale
            img = Image.open(image_path).convert('L')
            img_np = np.array(img, 'uint8')

            # Extract user_id from filename
            filename = os.path.split(image_path)[1]  # e.g., "Alishba.1.1.jpg"
            user_id = int(filename.split(".")[1])

            faces.append(img_np)
            ids.append(user_id)
        except Exception as e:
            print(f"Skipping {image_path}: {e}")

    if len(faces) == 0:
        print("No valid images to train.")
        return

    ids = np.array(ids)

    # Create LBPH face recognizer
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    print("Training complete! Classifier saved as 'classifier.xml'.")

# Train the classifier using all images in the 'data' folder
train_classifier("data")


Training complete! Classifier saved as 'classifier.xml'.


Face detection/recognition


In [26]:
import cv2
import numpy as np
from PIL import Image
import os


In [73]:
import cv2

# ------------------- Mapping IDs to Names ------------------- #
id_to_name = {
    1: "Alishba",
    2: "Kinza",
    3: "Maryam",
}

# ------------------- Function to Draw Boundaries ------------------- #
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, clf, id_to_name):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    
    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        
        # Predict ID using trained classifier
        id, pred = clf.predict(gray_img[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))
        
        # Display name or UNAUTHORIZED
        if confidence > 80:
            name = id_to_name.get(id, "Unknown")
            cv2.putText(img, f"{name} ({confidence}%)", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        else:
            cv2.putText(img, "UNAUTHORIZED", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    
    return img

# ------------------- Load Haar Cascade & Classifier ------------------- #
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
if faceCascade.empty():
    raise Exception("Error: Could not load Haar cascade xml file.")

clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# ------------------- Open Webcam ------------------- #
video_capture = cv2.VideoCapture(0)
if not video_capture.isOpened():
    raise Exception("Error: Could not open webcam.")

# ------------------- Real-Time Face Recognition ------------------- #
while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to grab frame from webcam.")
        break

    frame = draw_boundary(frame, faceCascade, 1.3, 5, (0, 255, 0), clf, id_to_name)
    cv2.imshow("Face Recognition", frame)
    
    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
